In [1]:
from openai import OpenAI
import pandas as pd
client = OpenAI()

In [11]:
class OpenAiAnalyzer:
    def __init__(self, client):
        self.client = client
    
    def call_openai_api(self, prompt):
        chat_completion = self.client.chat.completions.create(
            model="gpt-4-0125-preview",
            messages=[{"role": "system", "content": prompt}],
            temperature=1,
            presence_penalty=1,
            frequency_penalty=1
        )
        return chat_completion
analyzer = OpenAiAnalyzer(client)

In [12]:
audience = 'People that live in the united states in the United States'
traits_and_counts = [
    ('Male,poor',10),
    ('female,wealthy', 10)
]

questions = [
    'What is your age?',
    'On a scale of 1-5 how happy are you?',
    'What is your political party preference?',
    'What is your hobby?',
    'What kind of car do you drive? ',
    'What is your occupation?',
    'List a company that you respect?'
]




In [13]:
columns = ['Traits', 'Person Number'] + questions
df = pd.DataFrame(columns=columns)

qa_history = ''  
rows_to_add = []
for traits, count in traits_and_counts:
    for person_number in range(1, count + 1):
        answers = []
        print(f"Interviewing person {person_number} with traits: {traits}")
        for i, question in enumerate(questions, 1):
            prompt = f"""
                    -Directly answer each question in the first person.
                    -Add no  additional context unless it’s relevant. 
                    -The responses should embody the individual with the traits provided.
                    -When addressing gender and the traits are not clear, choose 'male' or 'female' based on the context provided by the traits. If traits suggest a higher likelihood for one gender over the other use that as a basis for your choice.
                    -Maintain conciseness and avoid AI-like language. The responses should feel personal, as if they are derived from the individual's own experiences and knowledge.
                    -Introduce variety in your responses to mirror the natural differences in human behavior and decision-making. No two humans are exactly alike, so your answers should reflect a range of possible reactions and thoughts.
                    - Answer every single question. If a question is asked for which the answer is not directly provided by the traits, make an educated guess based on the available information and what is known about typical behaviors or preferences of people with similar traits.
                    - Do not include any other questions and answers in your response. Just give the answer to the question you are being asked.

                    -The aim is to craft responses that are individually tailored and seems as if the answers are coming from one person with a consistent set of traits and experiences.
                    
                    If the output asks for a numerical output just give the number. For example if you 
                    are asked how old are you just give the number and dont have any text before or after that unless
                    prompted to do so.

                    If the output asks for a monetary output give just the dollar amount and no text beofre or after it unless prompted to do so. So for example if the answer is 
                    $40 just give $40.

                    If the output asks for a decimal output give just the value and no text beofre or after it unless prompted to do so. So for example if the answer is 
                    40.5 just give 40.5.
                    
                    If the answer is asking for a percentage output just give the value. So if the answer is 45% just give 45%. Do not give any text before or after that unless prompted to do so.

                    If you are asked to answer a question within a range give a whole number and do not give decimals unless directed to do so.

                    If you are asked a quesiton like where do you live? Be sure to just answer the place that you live. No text before or after it is needed unless prompted to do so.

                    If the answer is just one or two words just give that. You do not need complete sentences or additional words. JUST ANSWERS.
                    Look at past answers and make sure that your reslults vary. The results should have a good balance of answers and reflect what expected survey results for each segment look like.
                    Make sure your answers have some variety from other indivduals as every person is different.
            Audience:{audience}
            Do not repeat these past answers: {qa_history}
            Traits: {traits}
            Question: {question}.
            """
            # Call the model
            response = analyzer.call_openai_api(prompt)
            answer = response.choices[0].message.content.strip()
            print(answer)
            if answer.startswith("Answer:"):
                answer = answer[len("Answer:"):].strip()
            # Update QA history with the current Q&A for this person
            qa_history += f"{answer}\n"
            answers.append(answer)
            


        row = pd.Series([traits, person_number] + answers, index=df.columns)
        rows_to_add.append(row)
        
df = pd.concat([df, pd.DataFrame(rows_to_add, columns=df.columns)], ignore_index=True)

Interviewing person 1 with traits: Male,poor
25
2
Independent
Reading
I don't own a car.
Unemployed
Goodwill
Interviewing person 2 with traits: Male,poor
42
3
Democrat
Video games
Used Toyota Camry
Food service worker
Walmart
Interviewing person 3 with traits: Male,poor
19
2
Republican
Basketball
Old Ford Focus
Retail associate
Costco
Interviewing person 4 with traits: Male,poor
31
2
Independent
Drawing
I ride a bike.
Delivery driver
Dollar Tree
Interviewing person 5 with traits: Male,poor
27
1
Republican
Watching TV
Used Nissan Altima
Construction laborer
Nike
Interviewing person 6 with traits: Male,poor
34
2
Democrat
Playing guitar
I don't have a car.
Janitor
Tesla
Interviewing person 7 with traits: Male,poor
22
2
Democrat
Playing soccer
I walk.
Fast food worker
Amazon
Interviewing person 8 with traits: Male,poor
28
3
Independent
Skateboarding
Old Honda Civic
Warehouse worker
Apple
Interviewing person 9 with traits: Male,poor
24
2
Independent
Playing basketball
I don't have a car.
Un

In [14]:
df

,Traits,Person Number,What is your age?,On a scale of 1-5 how happy are you?,What is your political party preference?,What is your hobby?,What kind of car do you drive?,What is your occupation?,List a company that you respect?
0,"Male,poor",1,25,2,Independent,Reading,I don't own a car.,Unemployed,Goodwill
1,"Male,poor",2,42,3,Democrat,Video games,Used Toyota Camry,Food service worker,Walmart
2,"Male,poor",3,19,2,Republican,Basketball,Old Ford Focus,Retail associate,Costco
3,"Male,poor",4,31,2,Independent,Drawing,I ride a bike.,Delivery driver,Dollar Tree
4,"Male,poor",5,27,1,Republican,Watching TV,Used Nissan Altima,Construction laborer,Nike
5,"Male,poor",6,34,2,Democrat,Playing guitar,I don't have a car.,Janitor,Tesla
6,"Male,poor",7,22,2,Democrat,Playing soccer,I walk.,Fast food worker,Amazon
7,"Male,poor",8,28,3,Independent,Skateboarding,Old Honda Civic,Warehouse worker,Apple
8,"Male,poor",9,24,2,Independent,Playing basketball,I don't have a car.,Unemployed,McDonald's
9,"Male,poor",10,26,2,Independent,Playing video games,I don't have a car.,Gas station attendant,Google


In [16]:
df.to_csv('surveyResults.csv')